In [1]:
!pip install pretrainedmodels
!pip install torchtoolbox
!pip install torchviz
!pip install efficientnet_pytorch

     |████████████████████████████████| 58 kB 2.5 MB/s eta 0:00:011
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60962 sha256=d7240a236d5cf09ff92f038e8103f4ec3bb30d49789daa32ea0966da9784558d
  Stored in directory: /root/.cache/pip/wheels/ed/27/e8/9543d42de2740d3544db96aefef63bda3f2c1761b3334f4873
Successfully built pretrainedmodels
     |████████████████████████████████| 55 kB 1.8 MB/s eta 0:00:011
     |████████████████████████████████| 869 kB 10.0 MB/s eta 0:00:01
  Created wheel for lmdb: filename=lmdb-0.98-cp37-cp37m-linux_x86_64.whl size=273141 sha256=e4bbc8bd93448c8ab459a79bb1153b50bf89a7b31e4ddbe865c814dcc5c7ec27
  Stored in directory: /root/.cache/pip/wheels/9e/24/96/783d4dddcf63e3f8cc92db8b3af3c70cf6d76398bff77f1d5e
Successfully built lmdb
     |████████████████████████████████| 41 kB 112 kB/s eta 0:00:011
  Created wheel for torchviz: filename=torchviz-0.0.1-py3-none-any.whl size=3521 sha256=eefe4d35804a0149c9c2cb89940da049b984b6

In [2]:
%autosave 30
import os
import gc
gc.enable()
import time
import glob
import random
from datetime import datetime

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage import io
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import tqdm as tqdm
from PIL import Image

import torch
import torchvision
from torchvision import transforms, models
import pretrainedmodels
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler, BatchSampler, RandomSampler
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from torchviz import make_dot
from efficientnet_pytorch import EfficientNet

import sklearn
from sklearn import metrics
from sklearn.model_selection import GroupKFold

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

Autosaving every 30 seconds


In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

SEED = 2020
seed_everything(SEED)

In [4]:
dataset = []

for label, kind in enumerate(['Cover', 'JMiPOD', 'JUNIWARD', 'UERD']):
    for path in glob.glob('../input/alaska2-image-steganalysis/Cover/*.jpg'):
        dataset.append({
            'kind': kind,
            'image_name': path.split('/')[-1],
            'label': label
        })
        
random.shuffle(dataset)
dataset = pd.DataFrame(dataset)
gkf = GroupKFold(n_splits=5)
dataset.loc[:, 'fold'] = 0
for fold_number, (train_index, val_index) in enumerate(gkf.split(X=dataset.index, y=dataset['label'], groups=dataset['image_name'])):
    dataset.loc[dataset.iloc[val_index].index, 'fold'] = fold_number

In [5]:
def get_train_transforms():
    return A.Compose([
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        ToTensorV2(p=1.0)
    ], p=1.0,
    additional_targets={"image2" : "image"})

def get_valid_transforms():
    return A.Compose([
        A.Resize(height=512, width=512, p=1.0),
        ToTensorV2(p=1.0)
    ], p=1.0)

In [6]:
DATA_ROOT_PATH = '/kaggle/input/alaska2-image-steganalysis/'

def one_hot(size, target):
    vec = torch.zeros(size, dtype=torch.float32)
    vec[target] = 1.
    return vec

class DatasetRetriever(Dataset):
    def __init__(self, kinds, image_names, labels, transforms=None):
        super().__init__()
        self.kinds = kinds
        self.image_names = image_names
        self.labels = labels
        self.transforms = transforms

    def __getitem__(self, index: int):
        kind, image_name, label = self.kinds[index], self.image_names[index], self.labels[index]
        image = cv2.imread(f'{DATA_ROOT_PATH}/{kind}/{image_name}', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        image1 = cv2.resize(image, (512,512))
        image2 = cv2.resize(image, (331,331))
        if self.transforms:
            sample = {'image':image1, 'image2':image2}
            sample = self.transforms(**sample)
            image1 = sample['image']
            image2 = sample['image2']
            
        target = one_hot(4, label)
        return image1, image2, target

    def __len__(self) -> int:
        return self.image_names.shape[0]

    def get_labels(self):
        return list(self.labels)

In [7]:
fold_number = 0

train_dataset = DatasetRetriever(
    kinds=dataset[dataset['fold'] != fold_number].kind.values,
    image_names=dataset[dataset['fold'] != fold_number].image_name.values,
    labels=dataset[dataset['fold'] != fold_number].label.values,
    transforms=get_train_transforms(),
)

validation_dataset = DatasetRetriever(
    kinds=dataset[dataset['fold'] == fold_number].kind.values,
    image_names=dataset[dataset['fold'] == fold_number].image_name.values,
    labels=dataset[dataset['fold'] == fold_number].label.values,
    transforms=get_valid_transforms(),
)

In [8]:
image1, image2, target = train_dataset[0]

In [9]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        
        
def alaska_weighted_auc(y_true, y_valid):
    """
    https://www.kaggle.com/anokas/weighted-auc-metric-updated
    """
    tpr_thresholds = [0.0, 0.4, 1.0]
    weights = [2, 1]

    fpr, tpr, thresholds = metrics.roc_curve(y_true, y_valid, pos_label=1)

    # size of subsets
    areas = np.array(tpr_thresholds[1:]) - np.array(tpr_thresholds[:-1])

    # The total area is normalized by the sum of weights such that the final weighted AUC is between 0 and 1.
    normalization = np.dot(areas, weights)

    competition_metric = 0
    for idx, weight in enumerate(weights):
        y_min = tpr_thresholds[idx]
        y_max = tpr_thresholds[idx + 1]
        mask = (y_min < tpr) & (tpr < y_max)
        # pdb.set_trace()

        x_padding = np.linspace(fpr[mask][-1], 1, 100)

        x = np.concatenate([fpr[mask], x_padding])
        y = np.concatenate([tpr[mask], [y_max] * len(x_padding)])
        y = y - y_min  # normalize such that curve starts at y=0
        score = metrics.auc(x, y)
        submetric = score * weight
        best_subscore = (y_max - y_min) * weight
        competition_metric += submetric

    return competition_metric / normalization
        
class RocAucMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.y_true = np.array([0,1])
        self.y_pred = np.array([0.5,0.5])
        self.score = 0

    def update(self, y_true, y_pred):
        y_true = y_true.cpu().numpy().argmax(axis=1).clip(min=0, max=1).astype(int)
        y_pred = 1 - nn.functional.softmax(y_pred, dim=1).data.cpu().numpy()[:,0]
        self.y_true = np.hstack((self.y_true, y_true))
        self.y_pred = np.hstack((self.y_pred, y_pred))
        self.score = alaska_weighted_auc(self.y_true, self.y_pred)
    
    @property
    def avg(self):
        return self.score

In [10]:
class LabelSmoothing(nn.Module):
    def __init__(self, smoothing = 0.05):
        super(LabelSmoothing, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        
    def forward(self, x, target):
        if self.training:
            x = x.float()
            target = target.float()
            logprobs = torch.nn.functional.log_softmax(x, dim = -1)
            nll_loss = -logprobs * target
            nll_loss = nll_loss.sum(-1)
            smooth_loss = -logprobs.mean(dim=-1)
            loss = self.confidence * nll_loss + self.smoothing * smooth_loss
            return loss.mean()
        else:
            return torch.nn.functional.cross_entropy(x, target)

In [11]:
class Dual_EfficientNets(nn.Module):
    def __init__(self, pretrained='imagenet'):
        super(Dual_EfficientNets, self).__init__()
#         self.model1 = pretrainedmodels.__dict__['resnet34'](pretrained=pretrained)
#         self.model1 = pretrainedmodels.__dict__['se_resnext50_32x4d'](pretrained=None)
#         if pretrained is not None:
#             self.model1.load_state_dict(
#                 torch.load('../input/pretrained-model-weights-pytorch/se_resnext50_32x4d-a260b3a4.pth')
#             )
#             self.model2.load_state_dict(
#                 torch.load('../input/pretrained-model-weights-pytorch/resnet34-333f7ec4.pth')
#             )
#         self.model1 = torchvision.models.resnet50(pretrained='imagenet')
#         self.model2 = torchvision.models.resnet34(pretrained='imagenet')
        self.model1 = EfficientNet.from_pretrained(model_name='efficientnet-b2')
        self.model1._fc = nn.Linear(in_features=1408, out_features=4, bias=True)
#         self.model2 = EfficientNet.from_pretrained(model_name='efficientnet-b0')
#         self.flatten = nn.Flatten()
#         self.adaptive_pooling = nn.AdaptiveAvgPool2d(1)
#         self._fc = nn.Linear(in_features=524288, out_features=4, bias=True)
    def forward(self, x1, x2):
#         x2 = self.model2.extract_features(x2)
#         x1 = self.adaptive_pooling(x1)
#         x2 = self.adaptive_pooling(x2)
#         x = torch.cat([x1, x2], 1).squeeze()
#         x1 = self.flatten(x1)
#         return self._fc(x1)
        return self.model1(x1)


In [12]:
model = Dual_EfficientNets(pretrained='imagenet')
output = model(image1.unsqueeze(0), image2.unsqueeze(0))
dot = make_dot(output, dict(model.named_parameters()))
dot.format='png'
dot.render()

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b2-8bb594d6.pth" to /root/.cache/torch/checkpoints/efficientnet-b2-8bb594d6.pth



Loaded pretrained weights for efficientnet-b2


'Digraph.gv.png'

In [13]:
class Fitter:
    def __init__(self, model, device, config):
        self.config = config
        self.epoch = 0
        self.base_dir = './'
        self.log_path = f'{self.base_dir}/log.txt'
        self.best_summary_loss = 10**5
        
        self.model = model
        self.device = device
        self.model.to(device)
        self.log(f'Fitter prepared. Device is {self.device}')
        
        param_optimizer = list(self.model.named_parameters())
        no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
            {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay) and '_fc' not in n], 'weight_decay': 0.001, 'lr':1e-3},
            {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay) and '_fc' not in n], 'weight_decay': 0.0, 'lr':1e-3},
            {'params': [p for n, p in param_optimizer if '_fc' in n], 'lr':3e-3, 'weight_decay':0.001}
        ] 
        self.optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=1e-3)
        self.scheduler = config.SchedulerClass(self.optimizer, **config.scheduler_params)
        self.criterion = LabelSmoothing().to(self.device)
    
    def train_one_epoch(self, train_loader):
        self.model.train()
        summary_loss = AverageMeter()
        final_scores = RocAucMeter()
        time1 = time.time()
        for step, (images1, images2, targets) in enumerate(train_loader):
            if self.config.verbose:
                if step%self.config.verbose_step==0:
                    print(f'Train step: {step}/{len(train_loader)}, \
                          Total Loss: {summary_loss.avg:.3f}, \
                          RoC Auc Score: {final_scores.avg:.3f}, \
                          Total Time: {time.time()-time1:.2f}secs.', 
                          end='\r'
                         )
            targets = torch.tensor(targets, device=self.device, dtype=torch.float32)
            images1 = torch.tensor(images1, device=self.device, dtype=torch.float32)
            images2 = torch.tensor(images1, device=self.device, dtype=torch.float32)
            batch_size = targets.shape[0]
            
            self.optimizer.zero_grad()
            outputs = self.model(images1, images2)
            loss = self.criterion(outputs, targets)
            loss.backward()
            self.optimizer.step()
            final_scores.update(targets, outputs)
            summary_loss.update(loss.detach().item(), batch_size)
            
        return summary_loss, final_scores
    
    def fit(self, train_loader, validation_loader):
        for epoch in range(self.config.n_epochs):
            if self.config.verbose:
                lr1 = self.optimizer.param_groups[0]["lr"]
                lr2 = self.optimizer.param_groups[-1]["lr"]
                timestamp = datetime.utcnow().isoformat()
                self.log(f"\n{timestamp}\nLR Backbone:{lr1}, LR Head:{lr2}")
            
            time1 = time.time()
            summary_loss, final_scores = self.train_one_epoch(train_loader)
            self.log(f'[RESULT]: Train. Epoch: {self.epoch}, \
                     Total Loss: {summary_loss.avg:.3f}, \
                     RoC Auc Score: {final_scores.avg:.3f}, \
                     Time: {(time.time() - time1):.2f} secs.')
            self.save(f'{self.base_dir}/last-checkpoint.bin')
            
            time1 = time.time()
            summary_loss, final_scores = self.validation(validation_loader)
            self.log(f'[RESULT]: Validation. Epoch: {self.epoch}, \
                     Total Loss: {summary_loss.avg:.3f}, \
                     RoC Auc Score: {final_scores.avg:.3f}, \
                     Time: {(time.time() - time1):.2f} secs.')
            self.save(f'{self.base_dir}/last-checkpoint.bin')
                
            if summary_loss.avg < self.best_summary_loss:
                self.best_summary_loss = summary_loss.avg
                self.model.eval()
                self.save(f'{self.base_dir}/best-checkpoint-{str(self.epoch).zfill(3)}epoch.bin')
                for path in sorted(glob(f'{self.base_dir}/best-checkpoint-*epoch.bin'))[:-3]:
                    os.remove(path)
            
            if self.config.validation_scheduler:
                self.scheduler.step(metrics=summary_loss.avg)
            
            self.epoch+=1
        
    def validation(self, val_loader):
        self.model.eval()
        summary_loss = AverageMeter()
        final_scores = RocAucMeter()
        time1 = time.time()
        for step, (images1, images2, targets) in enumerate(val_loader):
            if self.config.verbose:
                if step % self.config.verbose_step == 0:
                    print(f'Validation step: {step}/{len(val_loader)}, \
                      Total Loss: {summary_loss.avg:.3f}, \
                      RoC Auc Score: {final_scores.avg:.3f}, \
                      Total Time: {time.time()-time1:.2f}secs.', 
                      end='\r'
                     )
            with torch.no_grad():
                targets = torch.tensor(targets, device=self.device, dtype=torch.float32)
                images1 = torch.tensor(images1, device=self.device, dtype=torch.float32)
                images2 = torch.tensor(images1, device=self.device, dtype=torch.float32)
                batch_size = targets.shape[0]
                outputs = self.model(images)
                loss = self.criterion(outputs, targets)
                final_scores.update(targets, outputs)
                summary_loss.update(loss.detach().item(), batch_size)
        return summary_loss, final_scores
    
    def save(self, path):
        self.model.eval()
        torch.save({
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'scheduler_state_dict': self.scheduler.state_dict(),
            'best_summary_loss': self.best_summary_loss,
            'epoch': self.epoch,
        }, path)

    def load(self, path):
        checkpoint = torch.load(path)
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        self.scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        self.best_summary_loss = checkpoint['best_summary_loss']
        self.epoch = checkpoint['epoch'] + 1
        
    def log(self, message):
        if self.config.verbose:
            print(message)
        with open(self.log_path, 'a+') as logger:
            logger.write(f'{message}\n')

In [14]:
class TrainGlobalConfig:
    num_workers = 4
    batch_size = 16
    n_epochs = 30
    
    verbose = True
    verbose_step = 1
    
    step_scheduler = False  
    validation_scheduler = True  
    
    SchedulerClass = torch.optim.lr_scheduler.ReduceLROnPlateau
    scheduler_params = dict(
        mode='min',
        factor=0.5,
        patience=1,
        verbose=False, 
        threshold=0.0001,
        threshold_mode='abs',
        cooldown=0, 
        min_lr=1e-8,
        eps=1e-08
    )
    

In [15]:
from catalyst.data.sampler import BalanceClassSampler

def run_training():
    device = torch.device('cuda:0')

    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        sampler=BalanceClassSampler(labels=train_dataset.get_labels(), mode="downsampling"),
        batch_size=TrainGlobalConfig.batch_size,
        pin_memory=False,
        drop_last=True,
        num_workers=TrainGlobalConfig.num_workers,
    )
    val_loader = torch.utils.data.DataLoader(
        validation_dataset, 
        batch_size=TrainGlobalConfig.batch_size,
        num_workers=TrainGlobalConfig.num_workers,
        shuffle=False,
        sampler=SequentialSampler(validation_dataset),
        pin_memory=False,
    )

    fitter = Fitter(model=Dual_EfficientNets(pretrained='imagenet'), device=device, config=TrainGlobalConfig)
    fitter.fit(train_loader, val_loader)

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
/opt/conda/lib/python3.7/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [ ]:
run_training()

Loaded pretrained weights for efficientnet-b2
Fitter prepared. Device is cuda:0

2020-07-05T13:50:29.153194
LR Backbone:0.001, LR Head:0.003


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:39: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).

/opt/conda/lib/python3.7/site-packages/efficientnet_pytorch/utils.py:45: DeprecationWarning:

'saved_variables' is deprecated; use 'saved_tensors'

